# 4 : Canny Edge Detector 

In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
import cv2
#load image
img = cv2.imread("/Users/Matt Hetrick/Desktop/COS429/puppy_training.jpg")

In [ ]:
# make floating point 
# convert to greyscale 
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#img_gray = img_gray[190-100:190+100,380-100:380+100] # 200x200 grid wanted to extract?
img_gray = img_gray / 255.0

In [ ]:
[Fx, Fy] = filteredGradient(img_gray, 1)
[F, D] = edgeStrengthAndOrientation(Fx, Fy)

In [ ]:
I = suppression(F, D)

In [ ]:
edgeMap = hysteresisThresholding(I, D, .2, .4)

In [ ]:
dimensions = img_gray.shape 
# height,width, 
height = img_gray.shape[0]
width = img_gray.shape[1]
print("height:", height)
print("width:", width)

In [ ]:
cv2.imshow('grey 200x200 pixel image', img_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
sigma = 1
x = np.linspace(-3*sigma,3*sigma,6*sigma+1)
y = np.linspace(-3*sigma,3*sigma,6*sigma+1)
kernel_x = (-x/(2*np.pi*sigma**4))*np.exp(-x/(sigma**2))
kernel_y = (-y/(2*np.pi*sigma**4))*np.exp(-y/(sigma**2))

In [ ]:
kernel_x = np.expand_dims(kernel_x, axis=1)
kernel_y = np.expand_dims(kernel_y, axis=0)

Fx = cv2.filter2D(img_gray,-1,kernel_x)
Fy = cv2.filter2D(img_gray,-1,kernel_y)

In [ ]:
np.arctan([Fx/Fy])

### Final function ---  filteredGradient()

In [2]:
def filteredGradient(im, sigma):        
    sigma = 1
    x = np.linspace(-3*sigma,3*sigma,6*sigma+1)
    y = np.linspace(-3*sigma,3*sigma,6*sigma+1)
    kernel_x = (-x/(2*np.pi*sigma**4))*np.exp(-x/(sigma**2))
    kernel_y = (-y/(2*np.pi*sigma**4))*np.exp(-y/(sigma**2))
    
    kernel_x = np.expand_dims(kernel_x, axis=1)
    kernel_y = np.expand_dims(kernel_y, axis=0)

    Fx = cv2.filter2D(im,-1,kernel_x)
    Fy = cv2.filter2D(im,-1,kernel_y)

    return Fx, Fy

### edge strength 

In [ ]:
Fx = cv2.filter2D(img_gray,-1,kernel_x)
Fy = cv2.filter2D(img_gray,-1,kernel_y)
D = np.arctan([Fy/Fx])
#outputs a (1, 200, 200) shape so remove the first column
D = D[0,:,:]
F = np.sqrt(Fy**2 + Fx**2) 

In [ ]:
import pandas as pd
F_df = pd.DataFrame(F)
F_fx = pd.DataFrame(Fx)
F_fy = pd.DataFrame(Fy)

In [3]:
def edgeStrengthAndOrientation(Fx, Fy):
    # Given horizontal and vertical gradients for an image, computes the edge
    # strength and orientation images.
    #
    # Fx: 2D double array with shape (height, width). The horizontal gradients.
    # Fy: 2D double array with shape (height, width). The vertical gradients.

    # Returns:
    # F: 2D double array with shape (height, width). The edge strength
    #        image.
    # D: 2D double array with shape (height, width). The edge orientation
    #        image.

    D = np.arctan([Fy/Fx])
    #outputs a (1, 200, 200) shape so remove the first column
    D = np.squeeze(D, axis = 0)
    F = np.sqrt(Fy**2 + Fx**2) 
    
    return F, D

### NonMax Supression 

In [4]:
def suppression(F, D):
    
    # get principal directions
    D_star = D
    height = D.shape[0]
    width = D.shape[1]
    for i in range(height):
        for j in range(width):
            if (D[i][j] <=((math.pi)/8)):
                D_star[i][j] = 0
            elif (D[i][j] <=(3*(math.pi)/8)):
                D_star[i][j] = (math.pi/4)
            elif (D[i][j] <=(5*(math.pi)/8)):
                D_star[i][j] = (math.pi/2)
            else: 
                D_star[i][j] = (3*math.pi/4)
            
    # pad columns with 0s
    F_new = np.insert(F, 0, 0, axis=1)
    F_new = np.append(F_new, np.zeros((F_new.shape[0],1)), axis=1)
    # pad rows with 0s
    F_new = np.insert(F_new, 0, 0, axis=0)
    F_new = np.append(F_new, np.zeros((1,F_new.shape[1])), axis=0)
            
    I = np.zeros((height, width))        
    def suppress(D_star, F, I):
        starheight = D_star.shape[0]
        starwidth = D_star.shape[1]
        for i in range(starheight):
            for j in range(starwidth):
                if (D_star[i][j] == 0):
                    if (F[i][j] < F[i+1][j]) or (F[i,j] < F[i-1][j]):
                        I[i][j] = 0
                    else: I[i][j] = F[i][j]
                elif (D_star[i,j] == ((math.pi)/4)):
                    if (F[i][j] < F[i+1][j+1]) or (F[i][j] < F[i-1][j-1]):
                        I[i][j] = 0
                    else: I[i][j] = F[i][j]
                elif (D_star[i][j] == ((math.pi)/2)):
                    if (F[i][j] < F[i][j+1]) or (F[i][j] < F[i][j-1]):
                        I[i][j] = 0
                    else: I[i][j] = F[i][j]
                else:
                    if (F[i][j] < F[i-1][j+1]) or (F[i][j] < F[i+1][j-1]):
                        I[i][j] = 0
                    else: I[i][j] = F[i][j]
        return I 
    
    I = suppress(D_star,F_new,I)  
        
    return I

In [ ]:
D_star = D
height = D.shape[0]
width = D.shape[1]
for i in range(height):
    for j in range(width):
        if (D[i][j] <=((math.pi)/8)):
            D_star[i][j] = 0
        elif (D[i][j] <=(3*(math.pi)/8)):
            D_star[i][j] = (math.pi/4)
        elif (D[i][j] <=(5*(math.pi)/8)):
            D_star[i][j] = (math.pi/2)
        else: 
            D_star[i][j] = (3*math.pi/4)

In [ ]:
# pad columns with 0s
F_new = np.insert(F, 0, 0, axis=1)
F_new = np.append(F_new, np.zeros((F_new.shape[0],1)), axis=1)
# pad rows with 0s
F_new = np.insert(F_new, 0, 0, axis=0)
F_new = np.append(F_new, np.zeros((1,F_new.shape[1])), axis=0)

In [ ]:
#maybe make this a function because we have to use logic again in thresholding
def suppress(D_star, F, img_gray):
    starheight = D_star.shape[0]
    starwidth = D_star.shape[1]
    I = np.empty((starheight, starwidth))
    for i in range(starheight):
        for j in range(starwidth):
            if (D_star[i][j] == 0):
                if (F[i][j] < F[i+1][j]) or (F[i,j] < F[i-1][j]):
                    I[i][j] = 0
                else: I[i][j] = F[i][j]
            elif (D_star[i,j] == ((math.pi)/4)):
                if (F[i][j] < F[i+1][j+1]) or (F[i][j] < F[i-1][j-1]):
                    I[i][j] = 0
                else: I[i][j] = F[i][j]
            elif (D_star[i][j] == ((math.pi)/2)):
                if (F[i][j] < F[i][j+1]) or (F[i][j] < F[i][j-1]):
                    I[i][j] = 0
                else: I[i][j] = F[i][j]
            else:
                if (F[i][j] < F[i-1][j+1]) or (F[i][j] < F[i+1][j-1]):
                    I[i][j] = 0
                else: I[i][j] = F[i][j]
                
    return I 
img_g = suppress(D_star,F_new,img_gray)    

In [ ]:
cv2.imshow('grey 200x200 pixel image', img_g)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Hysteresis Thresholding

In [5]:
def hysteresisThresholding(I, D, tL, tH):
    
    height = I.shape[0]
    width = I.shape[1]
    # need to normalize heights
    def normalize(I):
        mini = I.min()
        maxi = I.max()
        I_n = (I - mini) / (maxi - mini)
        return I_n
    I_norm = normalize(I) 
    
    pixList = []  
    # we could do a bfs here and trace out the edges as we search
    for i in range(height):
        for j in range(width):
            if (I_norm[i][j] > tH): 
                pixList.append([i,j])
                        
    # recalculate rounded directions            
    D_star = D
    height = D.shape[0]
    width = D.shape[1]
    for i in range(height):
        for j in range(width):
            if (D[i][j] <=((math.pi)/8)):
                D_star[i][j] = 0
            elif (D[i][j] <=(3*(math.pi)/8)):
                D_star[i][j] = (math.pi/4)
            elif (D[i][j] <=(5*(math.pi)/8)):
                D_star[i][j] = (math.pi/2)
            else: 
                D_star[i][j] = (3*math.pi/4)
                        
                        
    I_new = np.insert(I_norm, 0, 0, axis=1)
    I_new = np.append(I_new, np.zeros((I_new.shape[0],1)), axis=1)
    # pad rows with 0s
    I_new = np.insert(I_new, 0, 0, axis=0)
    I_new = np.append(I_new, np.zeros((1,I_new.shape[1])), axis=0)   
    
    newList = []                    
    for pixel in pixList:
        i = pixel[0]
        j = pixel[1]
        if (D_star[i][j] == 0):
            if (tL < I_new[i+1][j]): 
                newList.append([i+1,j])
            if (tL < I_new[i-1][j]):
                newList.append([i-1,j])
        elif (D_star[i,j] == ((math.pi)/4)):
            if (tL < I_new[i+1][j+1]): 
                newList.append([i+1, j+1])                       
            if (tL < I_new[i-1][j-1]):
                newList.append([i-1,j-1])
        elif (D_star[i][j] == ((math.pi)/2)):
            if (tL < I_new[i][j+1]):
                newList.append([i, j+1])
            if (tL < I_new[i][j-1]):
                newList.append([i, j-1])
        elif (D_star[i][j] == ((3*(math.pi))/4)):
            if (tL < I_new[i-1][j+1]):
                newList.append([i-1, j+1])
            if (tL < I_new[i+1][j-1]):
                newList.append([i+1, j-1])
    
    edgeMap = np.zeros((height, width))
    for i in range(height):
        for j in range(width):            
            if ([i,j] in newList):
                edgeMap[i][j] = 1
                
    return edgeMap

In [ ]:
height = img_g.shape[0]
width = img_g.shape[1]
# need to normalize heights
def normalize(I):
    height = I.shape[0]
    width = I.shape[1]
    mini = I.min()
    maxi = I.max()
    I_n = (I - mini) / (maxi - mini)
    return I_n
I_norm = normalize(img_g)    

In [ ]:
I_norm.max()

In [ ]:
pixList = []  
tH = 1.5*I_norm.mean()
tL = 1.25*I_norm.mean()
# we could do a bfs here and trace out the edges as we search
for i in range(height):
    for j in range(width):
        if (I_norm[i][j] > tH): 
            pixList.append([i,j])

In [ ]:
I_new = np.insert(I_norm, 0, 0, axis=1)
I_new = np.append(I_new, np.zeros((I_new.shape[0],1)), axis=1)
# pad rows with 0s
I_new = np.insert(I_new, 0, 0, axis=0)
I_new = np.append(I_new, np.zeros((1,I_new.shape[1])), axis=0)  

In [ ]:
newList = []                    
for pixel in pixList:
    i = pixel[0]
    j = pixel[1]
    if (D_star[i][j] == 0):
        if (tL < I_new[i+1][j]): 
            newList.append([i+1,j])
        if (tL < I_new[i-1][j]):
            newList.append([i-1,j])
    elif (D_star[i,j] == ((math.pi)/4)):
        if (tL < I_new[i+1][j+1]): 
            newList.append([i+1, j+1])                       
        if (tL < I_new[i-1][j-1]):
            newList.append([i-1,j-1])
    elif (D_star[i][j] == ((math.pi)/2)):
        if (tL < I_new[i][j+1]):
            newList.append([i, j+1])
        if (tL < I_new[i][j-1]):
            newList.append([i, j-1])
    elif (D_star[i][j] == ((3*(math.pi))/4)):
        if (tL < I_new[i-1][j+1]):
            newList.append([i-1, j+1])
        if (tL < I_new[i+1][j-1]):
            newList.append([i+1, j-1])

In [ ]:
for i in range(height):
    for j in range(width):            
        if ([i,j] in newList):
            img_gray[i][j] = 1
        else:
            img_gray[i][j] = 0

In [ ]:
cv2.imshow('grey 200x200 pixel image', img_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
def cannyEdgeDetection(im, sigma, tL, tH):
    # im: 2D double array with shape (height, width). The input image.
    # sigma: double. The standard deviation of the gaussian blur kernel.
    # tL: double. The low threshold for detection.
    # tH: double. The high threshold for detection.

    # Returns:
    # edgeMap: 2D binary image of shape (height, width). Output edge map, where edges are 1 and other pixels are 0.
    
    #make floating point and convert to grayscale
    img_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    img_gray = img_gray / 255.0
    
    [Fx, Fy] = filteredGradient(img_gray, sigma)
    [F, D] = edgeStrengthAndOrientation(Fx, Fy)
    I = suppression(F, D)
    edgeMap = hysteresisThresholding(I, D, tL, tH)
  
    return edgeMap

In [8]:
r = cannyEdgeDetection(img, 1, .2, .4)

In [9]:
cv2.imshow('dog', r)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_gray = img_gray / 255.0
[Fx, Fy] = filteredGradient(img_gray, 1)
[F, D] = edgeStrengthAndOrientation(Fx, Fy)
I = suppression(F, D)

In [14]:
def normalize(I):
    height = I.shape[0]
    width = I.shape[1]
    mini = I.min()
    maxi = I.max()
    I_n = (I - mini) / (maxi - mini)
    return I_n
I_norm = normalize(I) 

In [15]:
1.25*(I_norm.mean())

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.27205805, 0.26759845,
        0.26413455],
       [0.        , 0.        , 0.43805973, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.59473932, 0.        , ..., 0.6760013 , 0.80577922,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.6993772 , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])